# Photovoltaics Interactive Tabletop
## Cristian Sorescu 879091

In [80]:
import numpy as np
import pandas as pd
import tsfresh
import matplotlib.pyplot as plt

# Patterns data
gestures = []

# fist formation gesture
handDetect = pd.read_csv('csv/hand_detected_1.csv', delim_whitespace=False, header=[0, 1])
gestures.append([handDetect, 'Hand Detetction'])
# fist formation gesture
fistForm = pd.read_csv('csv/fist_form_1.csv', delim_whitespace=False, header=[0, 1], index_col=0)
gestures.append([fistForm, 'Fist Formation'])
# clockwise motion gesture
clockwise = pd.read_csv('csv/clockwise_1.csv', delim_whitespace=False, header=[0, 1], index_col=0)
gestures.append([clockwise, 'Clockwise Motion'])
# counterclockwise motion gesture
counterclockwise = pd.read_csv('csv/counterclockwise_2.csv', delim_whitespace=False, header=[0, 1], index_col=0)
gestures.append([counterclockwise, 'Counterclockwise Motion'])
# continuous swipes motion gesture
swipes = pd.read_csv('csv/swipe_1.csv', delim_whitespace=False, header=[0, 1], index_col=0)
gestures.append([swipes, 'Continuous Swipes'])
# singular swipe motion gesture
swipe = pd.read_csv('csv/swipes_1.csv', delim_whitespace=False, header=[0, 1], index_col=0)
gestures.append([swipe, 'Singular Swipe'])
# raise and Lower hand motion
raiseLower = pd.read_csv('csv/raiseLower_1.csv', delim_whitespace=False, header=[0, 1], index_col=0)
gestures.append([raiseLower, 'Raise/Lower Hand Motion'])

In [ ]:
# plot the data

#fistForm.shape
#fistForm.head(3)
gestures[:5]

#handDetect['Channel A'].plot(linewidth=1);
#fistForm['Channel A'].plot(linewidth=1);
#clockwise['Channel A'].plot(linewidth=1);

In [ ]:
# extract features
fistForm.reset_index(inplace=True)
fistForm_features = tsfresh.extract_features(fistForm, column_id='index', n_jobs=4)

#fistForm_features = tsfresh.extract_features(fistForm)

#clockwise_features = tsfresh.extract_features(clockwise)

#counterclockwise_features = tsfresh.extract_features(counterclockwise)

#swipes_features = tsfresh.extract_features(swipes)

#swipe_features = tsfresh.extract_features(swipe)

#raiseLower_features = tsfresh.extract_features(raiseLower)

In [ ]:
from tsfresh.utilities.dataframe_functions import impute

impute(handDetect_features)

In [ ]:
#handDetect_features.head()
#handDetect.resample('3T').sum()
#handDetect.describe()

In [ ]:
for gesture in gestures:
    gestureDF, title = gesture
    print(round(gestureDF.max().to_numpy()[0], 1))

In [ ]:
plt.rcParams.update({'font.size': 32})

for gesture in gestures:
    gestureDF, title = gesture
    #gestureDF.columns = gestureDF.columns.droplevel(1)
    plotTitle = title

    rollingMean = gestureDF.rolling(window = 100).mean()
    rollingMean = rollingMean.dropna().drop_duplicates()
    rollingStd = gestureDF.rolling(window = 12).std()
    #plt.plot(gestureDF, color = 'blue', label = 'Original')
    plt.plot(rollingMean, color = 'red', label = 'Rolling Mean')
    #plt.plot(rollingStd, color = 'black', label = 'Rolling STD')
    #plt.legend(loc = 'best', prop={'size': 13})
    #plt.title(plotTitle)
    plt.xlabel('Time (s)')
    plt.ylabel('Voltage (V)')
    #minR = round(gestureDF.min().to_numpy()[0], 1)
    #maxR = round(gestureDF.max().to_numpy()[0], 1)
    #plt.yticks(np.arange(minR, maxR, step=0.3)) 
    plt.show()

In [ ]:
import mass_ts as mts

ts = handDetect3.to_numpy()
q = handDetect.to_numpy()

print(handDetect3[:,0])

# mass
distances = mts.mass(ts[:,0], q[:,0])

# mass2
distances = mts.mass2(ts[:,0], q[:,0])

# mass3
#distances = mts.mass3(ts[:,0], q[:,0], 256)

# mass2_batch
# start a multi-threaded batch job with all cpu cores and give me the top 5 matches.
# note that batch_size partitions your time series into a subsequence similarity search.
# even for large time series in single threaded mode, this is much more memory efficient than
# MASS2 on its own.
batch_size = 10000
top_matches = 5
n_jobs = 3
indices, distances = mts.mass2_batch(ts[:,0], q[:,0], batch_size, 
    top_matches=top_matches, n_jobs=n_jobs)

# find minimum distance
min_idx = np.argmin(distances)

# find top 4 motif starting indices
k = 4
exclusion_zone = 25
top_motifs = mts.top_k_motifs(distances, k, exclusion_zone)

# find top 4 discord starting indices
k = 4
exclusion_zone = 25
top_discords = mts.top_k_discords(distances, k, exclusion_zone)

# Wavelet analysis

In [81]:
import pywt
x = handDetect.values
# Discrete Wavelet Transform
#cA, cD = pywt.dwt(x, 'db2')
#x2 = pywt.idwt(cA, cD, 'db2')
x

array([[0.00000000e+00, 1.14749600e-01],
       [3.33340000e-04, 9.76592300e-02],
       [6.66690000e-04, 1.01016300e-01],
       ...,
       [5.00049351e+00, 6.40888700e-02],
       [5.00082685e+00, 4.36414700e-02],
       [5.00116020e+00, 3.32651700e-02]])

In [16]:
# for mode_name in ['zero', 'constant', 'symmetric', 'reflect', 'periodic', 'smooth', 'periodization']:
#     mode = getattr(pywt.Modes, mode_name)
#     cA, cD = pywt.dwt(x, 'db2', mode)
#     print("Mode: %d (%s)" % (mode, mode_name))

Mode: 0 (zero)
Mode: 2 (constant)
Mode: 1 (symmetric)
Mode: 6 (reflect)
Mode: 4 (periodic)
Mode: 3 (smooth)
Mode: 5 (periodization)


In [82]:
wp = pywt.WaveletPacket2D(data=x, wavelet='db1', mode='symmetric')

In [86]:
wp.maxlevel

1

In [88]:
print(wp['a'].data)

[[0.10637109]
 [0.11298891]
 [0.1331874 ]
 ...
 [5.09976086]
 [5.06700978]
 [5.03944685]]


In [60]:
y = np.array([[1, 2, 3, 4, 5, 6, 7, 8]] * 8, 'd')
wp1 = pywt.WaveletPacket2D(data=y, wavelet='db1', mode='symmetric')
print(wp1['a'].data)

[[ 3.  7. 11. 15.]
 [ 3.  7. 11. 15.]
 [ 3.  7. 11. 15.]
 [ 3.  7. 11. 15.]]
